In [ ]:
#Papers:
## EvolveGCN
#https://jiechenjiechen.github.io/pub/evolvegcn.pdf
#Code:https://github.com/IBM/EvolveGCN
## Nature paper (Evolve + Attention)
#file:///C:/Users/Christian/Downloads/s41598-023-50977-6.pdf
## Efficent TGN
#https://dl.acm.org/doi/pdf/10.1145/3627673.3679104
#PyG
#https://pytorch-geometric.readthedocs.io/en/latest/index.html


In [ ]:
# Importing packages
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Home-made functions
from utils import year_to_t, unweight_adj, log_transform, exp_transform, get_link
from utils.plot import plot_loss, plot_roc

from make_data import load_edges, load_features, prepare_language, prepare_data
from model import EGCU_H
from mlp import LinkPredictorMLP
from training import train
from eval import eval_baseline, eval_model

In [ ]:
#Data Preparations

In [ ]:
# Data paths
PATH_NF = "./data/input/"
PATH_E = "./data/observation/"


# Data preperations
years = np.arange(1992,2019 + 1)
time_steps = len(years)
countries_codes = pd.read_csv(PATH_NF + "Country_codes_names.csv")
nodes = len(countries_codes)
countries = list(countries_codes["Name"])
codes = list(countries_codes["Code"])
country_number = countries_codes["No"].to_numpy() - 1
languages = list(pd.read_csv(PATH_NF + "Legend_Language.csv")["Language"])
# node id (number) -> Country name
dict_country = dict(zip(country_number,countries))
dict_code = dict(zip(country_number,codes))
dict_code_to_country = dict(zip(codes,countries))
dict_code_to_number = dict(zip(codes,country_number))
dict_code_to_language = dict(enumerate(languages))

In [ ]:
# Load features
#PATH_NF = "/kaggle/input/ds535-node-features-small/"
feature_names = ["GDP","GINI","HDI","PTS","U5M"]
X_np, non_nan_idx = load_features(PATH_node_features=PATH_NF,years=years,feature_names=feature_names, remove_nan = True)
n_to_country_non_nan = np.array(countries)[non_nan_idx]
# Load edges
#PATH_E = "/kaggle/input/ds535-refugee/"
A_np = load_edges(PATH_EDGES=PATH_E,log_transfrom_data=True, 
                  remove_self_loops=False, non_nan_idx=non_nan_idx, mode = "refugee", years_range = np.arange(1992,2019 + 1))
# Prepare language data
LAN_np = prepare_language(PATH=PATH_NF, non_nan_idx=non_nan_idx)
A_np.shape

In [ ]:
#Get test data
years1 = np.arange(1992,2020 + 1)
X_np1, non_nan_idx1 = load_features(PATH_node_features=PATH_NF,years=years1,feature_names=feature_names, remove_nan = True)
A_np1 = load_edges(PATH_EDGES=PATH_E,log_transfrom_data=True, 
                  remove_self_loops=False, non_nan_idx=non_nan_idx, mode = "refugee", years_range = years1)
# Prepare language data
LAN_np1 = prepare_language(PATH=PATH_NF, non_nan_idx=non_nan_idx1)

A1 = torch.from_numpy(A_np1)
X1 = torch.from_numpy(X_np1)

embedding_dim = 4
data1 = prepare_data(A1,X1, tt_idx = 28, embedding_dim=embedding_dim, 
                    LAN_np = LAN_np, weighted = False) # Set LAN_np = None to train without language embeddings
A_norm1,X_norm1,A_train1,A_valid,X_test1 = data1
A_valid.shape

In [ ]:
#Un-weighted learning (GCN_MA)

In [ ]:
#Hyperparameters Grid Search

output_dim = 16                 # Output dimension of GCN
num_gcn_layers = 2              # Number of GCN layers
num_heads = 4                   # Number of attention heads
loss_weights = [0.9,0.1]

A = torch.from_numpy(A_np)
X = torch.from_numpy(X_np)

tt_idx = 23
embedding_dim = 4
data = prepare_data(A,X, tt_idx = tt_idx, embedding_dim=embedding_dim,
                    LAN_np = LAN_np, weighted = False) # Set LAN_np = None to train without language embeddings
A_norm,X_norm,A_train,A_test,X_test = data

from itertools import product
import torch
from torch import nn, optim
from sklearn.model_selection import ParameterGrid

# Define the hyperparameter grid
param_grid = {
    'hidden_dim': [32, 64, 128],
#    'output_dim': [8, 16, 32],
#    'num_gcn_layers': [1, 2, 3],
#    'num_heads': [2, 4, 8],
    'learning_rate': [0.001, 0.0005, 0.0001],
    'edge_subset': [100, 200, 300],
#    'loss_weights': [[0.9, 0.1], [0.8, 0.2]],
}

grid = ParameterGrid(param_grid)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Placeholder for best parameters and lowest loss
best_params = None
lowest_loss = float('inf')

# Grid search
for params in grid:
    print(f"Testing configuration: {params}")
    
    # Unpack parameters
    hidden_dim = params['hidden_dim']
#    output_dim = params['output_dim']
#    num_gcn_layers = params['num_gcn_layers']
#    num_heads = params['num_heads']
    learning_rate = params['learning_rate']
    edge_subset = params['edge_subset']
#    loss_weights = params['loss_weights']

    # Initialize model and optimizer with the current set of hyperparameters
    GCN_MA = EGCU_H(
        input_dim=X_norm.shape[-1], 
        hidden_dim=hidden_dim, 
        output_dim=output_dim, 
        num_gcn_layers=num_gcn_layers, 
        num_heads=num_heads, 
        attention=True
    ).to(device)

    link_predictor = LinkPredictorMLP(input_dim=output_dim, weighted=False).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(
        list(GCN_MA.parameters()) + list(link_predictor.parameters()), 
        lr=learning_rate
    )

    # Train the model
    losses = train(
        X_norm=X_norm, A_norm=A_norm, A_train=A_train,
        model=GCN_MA, link_predictor=link_predictor,
        criterion=criterion, optimizer=optimizer, device=device,
        num_epochs=50,  # Use fewer epochs during grid search for efficiency
        edge_subset=edge_subset, loss_weights=loss_weights,
        calc_reverse=True, save_at=10, weighted=False
    )
    
    # Evaluate the model
    _, eval_loss = eval_model(
        X_norm=X_norm, A_norm=A_norm, A_test=A_test,
        model=GCN_MA, link_predictor=link_predictor, 
        device=device, weighted=False
    )
    print(f"Evaluation loss for this configuration: {eval_loss}")

    primary_loss = eval_loss[0]  # Adjust the index based on your tuple structure
    print(f"Primary loss for this configuration: {primary_loss}")

    # Update best parameters if current configuration is better
    if primary_loss < lowest_loss:
        lowest_loss = primary_loss
        best_params = params
    
print("\nGrid Search Complete")
print(f"Best parameters: {best_params}")
print(f"Lowest loss: {lowest_loss}")


In [ ]:
#best_params = {'edge_subset' : 200, 'learning_rate' : 0.0001, 'hidden_dim' : 32 }

In [ ]:
#build best model

A = torch.from_numpy(A_np)
X = torch.from_numpy(X_np)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Train Test Index
# Now 80 % is training and 20 % is test
tt_idx = 23
embedding_dim = 4
data = prepare_data(A,X, tt_idx = tt_idx, embedding_dim=embedding_dim,
                    LAN_np = LAN_np, weighted = False) # Set LAN_np = None to train without language embeddings
A_norm,X_norm,A_train,A_test,X_test = data
## Hyperparameters
# Training
num_epochs = 1000                # Number of training epochs
edge_subset = best_params['edge_subset'] # best_params['edge_subset'] = 100    # Number of edges to sample during training
calc_reverse = True            # Include reverse order calc of pairs
learning_rate = best_params['learning_rate'] # best_params['learning_rate'] = 0.0001 
loss_weights = [0.9,0.1]        # Balance positive/negative loss
# Model
input_dim = X_norm.shape[-1]    # F: feature dimension
hidden_dim = best_params['hidden_dim'] # best_params['hidden_dim'] = 32
output_dim = 16                 # Output dimension of GCN
num_gcn_layers = 2              # Number of GCN layers
num_heads = 4                   # Number of attention heads

In [ ]:
# Initialize the model, loss, and optimizer
GCN_MA = EGCU_H(input_dim=input_dim, 
                hidden_dim=hidden_dim, 
                output_dim=output_dim, 
                num_gcn_layers=num_gcn_layers,
                num_heads = num_heads,
                attention = True)

# Set tensors to device
GCN_MA.to(device)

link_predictor = LinkPredictorMLP(input_dim=output_dim, weighted=False).to(device) 
criterion = nn.BCELoss()  # Cross-entropy loss for link prediction
optimizer = optim.Adam(list(GCN_MA.parameters()) + list(link_predictor.parameters()), lr=learning_rate)

In [ ]:
# Train model
losses = train(X_norm=X_norm,A_norm=A_norm,A_train=A_train,
               model=GCN_MA,link_predictor=link_predictor,
               criterion=criterion,optimizer=optimizer,device=device,
               num_epochs=num_epochs,edge_subset=edge_subset, loss_weights=loss_weights,
               calc_reverse=calc_reverse,save_at = 50, weighted = False)

In [ ]:
# Evaluation
A_pred1, (AUC, AP) = eval_model(X_norm1,A_norm1,A_valid,model=GCN_MA,link_predictor=link_predictor, device=device,weighted=False)


In [ ]:
plot_loss(losses)

In [ ]:
#Finding hubs
#average metrics across years

In [ ]:
import numpy as np
import pandas as pd

# Derive filtered_countries from dict_country and non_nan_idx
countries = list(dict_country.values())
countries = [country.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom") for country in countries]

if isinstance(non_nan_idx, np.ndarray) and non_nan_idx.dtype == np.bool_:
    non_nan_idx = np.where(non_nan_idx)[0]
filtered_countries = [countries[i] for i in non_nan_idx]


def compute_network_flow_metrics_extended(adjacency_matrix, countries):
    """
    Compute extended network flow metrics for a weighted adjacency matrix.

    Args:
        adjacency_matrix: Weighted adjacency matrix (2D NumPy array).
        countries: List of country names corresponding to the nodes.

    Returns:
        flow_metrics: Dictionary containing inflow, outflow hubs, and flow metrics.
        flow_heterogeneity: Global metric representing flow distribution disparity.
    """
    # Compute inflows (sum rows) and outflows (sum columns)
    inflows = adjacency_matrix.sum(axis=1)  # Row sums (weighted inflows)
    outflows = adjacency_matrix.sum(axis=0)  # Column sums (weighted outflows)

    # Compute unweighted inflows and outflows
    unweighted_matrix = (adjacency_matrix > 0).astype(int)
    inflows_unweighted = unweighted_matrix.sum(axis=1)  # Row sums (unweighted inflows)
    outflows_unweighted = unweighted_matrix.sum(axis=0)  # Column sums (unweighted outflows)

    # Flow Efficiency (weighted)
    flow_efficiency = inflows / (outflows + 1)  # Avoid division by zero

    # Net Flow (weighted)
    net_flow = inflows - outflows

    # Flow Ratio (weighted)
    flow_ratio = inflows / (inflows + outflows + 1)  # Avoid division by zero

    # Flow Density (weighted)
    max_possible_flow = adjacency_matrix.sum()  # Total flow in the network
    total_flow = inflows + outflows
    flow_density = total_flow / max_possible_flow

    # Flow Heterogeneity (Global metric, weighted)
    heterogeneity_index = 1 - (np.sum(total_flow**2) / (np.sum(total_flow)**2))

    # Normalize inflows and outflows for hub scores (weighted)
    max_inflow = inflows.max() if inflows.max() > 0 else 1
    max_outflow = outflows.max() if outflows.max() > 0 else 1
    normalized_inflows = inflows / max_inflow
    normalized_outflows = outflows / max_outflow

    # Normalize inflows and outflows for hub scores (unweighted)
    max_inflow_unweighted = inflows_unweighted.max() if inflows_unweighted.max() > 0 else 1
    max_outflow_unweighted = outflows_unweighted.max() if outflows_unweighted.max() > 0 else 1
    normalized_inflows_unweighted = inflows_unweighted / max_inflow_unweighted
    normalized_outflows_unweighted = outflows_unweighted / max_outflow_unweighted

    # Map results to country names
    flow_metrics = {
        "inflow_hubs_weighted": {countries[i]: normalized_inflows[i] for i in range(len(countries))},
        "outflow_hubs_weighted": {countries[i]: normalized_outflows[i] for i in range(len(countries))},
        "inflow_hubs_unweighted": {countries[i]: normalized_inflows_unweighted[i] for i in range(len(countries))},
        "outflow_hubs_unweighted": {countries[i]: normalized_outflows_unweighted[i] for i in range(len(countries))},
        "flow_efficiency": {countries[i]: flow_efficiency[i] for i in range(len(countries))},
        "flow_ratio": {countries[i]: flow_ratio[i] for i in range(len(countries))},
        "flow_density": {countries[i]: flow_density[i] for i in range(len(countries))},
        "net_flow": {countries[i]: net_flow[i] for i in range(len(countries))},
    }

    return flow_metrics, heterogeneity_index

# List of metrics to compute
metrics_list = [
    "inflow_hubs_weighted",
    "outflow_hubs_weighted",
    "inflow_hubs_unweighted",
    "outflow_hubs_unweighted",
    "flow_efficiency",
    "flow_ratio",
    "flow_density",
    "net_flow",
]

# Initialize data structures to store metrics over time
num_years = A_np.shape[0]  # Number of years
metrics_over_time = {
    metric: {country: [] for country in filtered_countries} for metric in metrics_list
}
heterogeneity_index_over_time = []

# Loop over each year to compute metrics
for t in range(num_years):
    adjacency_matrix_t = A_np[t, :, :]
    adjacency_matrix_t = np.nan_to_num(adjacency_matrix_t)  # Replace NaNs with 0

    # Compute metrics for the year
    flow_metrics_t, heterogeneity_index_t = compute_network_flow_metrics_extended(
        adjacency_matrix_t, filtered_countries
    )

    # Store metrics for each country
    for metric in metrics_list:
        for country in filtered_countries:
            value = flow_metrics_t[metric][country]
            metrics_over_time[metric][country].append(value)

    # Store heterogeneity index
    heterogeneity_index_over_time.append(heterogeneity_index_t)

# Compute average metrics over the years
average_metrics = {}
for metric in metrics_list:
    average_metrics[metric] = {}
    for country in filtered_countries:
        values = metrics_over_time[metric][country]
        average_value = np.mean(values)
        average_metrics[metric][country] = average_value

# Compute average heterogeneity index
average_heterogeneity_index = np.mean(heterogeneity_index_over_time)

# Display results for average weighted and unweighted hubs
print("\nTop 5 Weighted Inflow Hubs (Average over Years):")
for country, score in sorted(average_metrics["inflow_hubs_weighted"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print("\nTop 5 Weighted Outflow Hubs (Average over Years):")
for country, score in sorted(average_metrics["outflow_hubs_weighted"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print("\nTop 5 Unweighted Inflow Hubs (Average over Years):")
for country, score in sorted(average_metrics["inflow_hubs_unweighted"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print("\nTop 5 Unweighted Outflow Hubs (Average over Years):")
for country, score in sorted(average_metrics["outflow_hubs_unweighted"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print("\nTop 5 Countries by Flow Efficiency (Weighted) (Average over Years):")
for country, score in sorted(average_metrics["flow_efficiency"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print("\nTop 5 Countries by Flow Ratio (Average over Years):")
for country, score in sorted(average_metrics["flow_ratio"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print("\nTop 5 Countries by Flow Density (Average over Years):")
for country, score in sorted(average_metrics["flow_density"].items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{country}: {score:.4f}")

print(f"\nAverage Flow Heterogeneity Index (Global): {average_heterogeneity_index:.4f}")


In [ ]:
import matplotlib.pyplot as plt

def visualize_top_metrics(flow_metrics, title, metric_key, top_n=5):
    """
    Visualize the top countries for a given metric.
    
    Args:
        flow_metrics: Dictionary containing flow metrics.
        title: Title for the plot.
        metric_key: Key in the flow_metrics dictionary to visualize.
        top_n: Number of top countries to display.
    """
    # Sort and extract top N countries
    sorted_metrics = sorted(flow_metrics[metric_key].items(), key=lambda x: x[1], reverse=True)[:top_n]
    countries = [item[0] for item in sorted_metrics]
    scores = [item[1] for item in sorted_metrics]

    # Plot
    plt.figure(figsize=(5, 3))
    plt.barh(countries, scores, color='skyblue')
    plt.xlabel('Score')
    plt.title(title)
    plt.gca().invert_yaxis()  # Invert y-axis to have the highest score at the top
    
    # Despine
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    plt.show()

def visualize_bottom_metrics(flow_metrics, title, metric_key, bottom_n=5):
    """
    Visualize the least-performing countries for a given metric.
    
    Args:
        flow_metrics: Dictionary containing flow metrics.
        title: Title for the plot.
        metric_key: Key in the flow_metrics dictionary to visualize.
        bottom_n: Number of bottom countries to display.
    """
    # Sort and extract bottom N countries
    sorted_metrics = sorted(flow_metrics[metric_key].items(), key=lambda x: x[1])[:bottom_n]
    countries = [item[0] for item in sorted_metrics]
    scores = [item[1] for item in sorted_metrics]

    # Plot
    plt.figure(figsize=(5, 3))
    plt.barh(countries, scores, color='salmon')
    plt.xlabel('Score')
    plt.title(title)
    plt.gca().invert_yaxis()  # Invert y-axis to have the highest score at the top
    
    # Despine
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Visualize top metrics
visualize_top_metrics(average_metrics, "Top 5 Weighted Inflow Hubs", "inflow_hubs_weighted")
visualize_top_metrics(average_metrics, "Top 5 Weighted Outflow Hubs", "outflow_hubs_weighted")
visualize_top_metrics(average_metrics, "Top 5 Unweighted Inflow Hubs", "inflow_hubs_unweighted")
visualize_top_metrics(average_metrics, "Top 5 Unweighted Outflow Hubs", "outflow_hubs_unweighted")
visualize_top_metrics(average_metrics, "Top 5 Countries by Flow Efficiency", "flow_efficiency")
visualize_top_metrics(average_metrics, "Top 5 Countries by Flow Ratio", "flow_ratio")
visualize_top_metrics(average_metrics, "Top 5 Countries by Flow Density", "flow_density")

# Visualize bottom metrics
visualize_bottom_metrics(average_metrics, "Bottom 10 Countries by Flow Efficiency", "flow_efficiency", bottom_n=10)



In [ ]:
#T -> T+1 (28 -> 29) validation
countries = [country.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom") for country in countries]
filtered_countries = [country.replace("United Kingdom of Great Britain and Northern Ireland", "UK") for country in filtered_countries]
filtered_countries = [country.replace("United Kingdom", "UK") for country in filtered_countries]
filtered_countries = [country.replace("United States of America", "USA") for country in filtered_countries]
filtered_countries = [country.replace("Syrian Arab Republic", "Syria") for country in filtered_countries]
filtered_countries = [country.replace("Iran (Islamic Republic of)", "Iran") for country in filtered_countries]
filtered_countries = [country.replace("Venezuela (Bolivarian Republic of)", "Venezuela") for country in filtered_countries]


In [ ]:
PATH_E = "data/observation/"
years1 = np.arange(1992,2020 + 1)
feature_names = ["GDP","GINI","HDI","PTS","U5M"]
X_np1, non_nan_idx1 = load_features(PATH_node_features=PATH_NF,years=years1,feature_names=feature_names, remove_nan = True)
A_np1 = load_edges(PATH_EDGES=PATH_E,log_transfrom_data=False, 
                  remove_self_loops=False, non_nan_idx=non_nan_idx, mode = "refugee", years_range = years1)
# Prepare language data
LAN_np1 = prepare_language(PATH=PATH_NF, non_nan_idx=non_nan_idx1)

A1 = torch.from_numpy(A_np1)
X1 = torch.from_numpy(X_np1)

data1 = prepare_data(A1,X1, tt_idx = 28, embedding_dim=embedding_dim, 
                    LAN_np = LAN_np, weighted = False) # Set LAN_np = None to train without language embeddings
A_norm1,X_norm1,A_train1,A_valid,X_test1 = data1
print(A_norm1.shape)
A_pred1, (AUC1, AP1) = eval_model(X_norm1,A_norm1,A_valid,model=GCN_MA,link_predictor=link_predictor, device=device,weighted=False)
A_pred_denoised = A_pred1.clone()

In [ ]:
def unweight_adj(A):
    if isinstance(A, np.ndarray):
        A_unweighted = (A > 0).astype(int)
    elif isinstance(A,torch.Tensor):
        A_unweighted = (A > 0).long()
    else:
        print("Error: Array has to be either numpy array or torch tensor")
        A_unweighted = None
    return A_unweighted

In [ ]:
#Gravity model
A_np29_gravity29 = pd.read_csv("data/Refugee_Stock_2020_Gravity_Model.csv"
                              , index_col=0)
A_np29_gravity29 = A_np29_gravity29.fillna(0)

max_finite_value = A_np29_gravity29[A_np29_gravity29 != np.inf].max().max()
A_np29_gravity29 = A_np29_gravity29.replace([np.inf, -np.inf], max_finite_value)

A_np29_gravity29 = np.log(A_np29_gravity29.copy() + 1)

A_np29_gravity29.index = A_np29_gravity29.index - 1
A_np29_gravity29.columns = A_np29_gravity29.columns.astype(int) - 1

A_np29_gravity29 = A_np29_gravity29.iloc[:, non_nan_idx]  # Select columns
A_np29_gravity29 = A_np29_gravity29.iloc[non_nan_idx, :]  # Select rows

A_np29_gravity29 = A_np29_gravity29.to_numpy()

A_np29_gravity29 = unweight_adj(A_np29_gravity29)


In [ ]:
A_np29 = A_valid.cpu().detach().numpy()

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Binarize the ground truth graph
A_np29_binary = (A_np29 > 0).astype(int)  # Threshold for edge presence

# Flatten the matrices
y_true = A_np29_binary.flatten()  # Ground truth (binary)
y_score = A_np29_gravity29.flatten()  # Predicted values (probabilities)

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(4, 3))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--", lw=1)  # Diagonal line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Predicted vs Actual Graph")
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure A_pred_denoised is a NumPy array
A_pred_denoised_np = A_pred_denoised.cpu().detach().numpy()
A_valid_np = A_valid.cpu().detach().numpy()  # Convert A_valid to NumPy

A_valid_np = A_valid_np.squeeze(0)

# Compute metrics for the validation set
flow_metrics_valid, flow_heterogeneity_valid = compute_network_flow_metrics_extended(A_valid_np, filtered_countries)

# Compute metrics for the predicted graph
predicted_flow_metrics, predicted_flow_heterogeneity = compute_network_flow_metrics_extended(A_pred_denoised_np, filtered_countries)

# -----------------------------
# Helper function: Plot comparison for top 10 countries
# -----------------------------
def plot_comparison(original_data, predicted_data, title, ylabel):
    top_10 = sorted(original_data.items(), key=lambda x: x[1], reverse=True)[:10]  # Top 10 from original
    countries, original_values = zip(*top_10)  # Separate into country names and values
    predicted_values = [predicted_data[country] for country in countries]  # Get corresponding predicted values
    
    x = np.arange(len(countries))  # X positions
    width = 0.4  # Bar width

    plt.figure(figsize=(6, 3))
    plt.bar(x - width / 2, original_values, width, label="Validation")
    plt.bar(x + width / 2, predicted_values, width, label="Predicted")
    plt.xticks(x, countries, rotation=45, ha="right")
    plt.title(title)
    plt.ylabel(ylabel)
    plt.grid(False)  # Remove grid
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

# -----------------------------
# Correlation for each metric and plotting comparisons
# -----------------------------

metrics_to_compare = {
    "Inflow Hubs": "inflow_hubs_weighted",
    "Outflow Hubs": "outflow_hubs_weighted",
}

for metric_name, metric_key in metrics_to_compare.items():
    print(f"\nCorrelation for {metric_name}:")
    corr = np.corrcoef(
        list(flow_metrics_valid[metric_key].values()),
        list(predicted_flow_metrics[metric_key].values())
    )[0, 1]
    print(f"Correlation: {corr:.4f}")

    plot_comparison(
        flow_metrics_valid[metric_key],
        predicted_flow_metrics[metric_key],
        f"{metric_name} Comparison (Top 10)",
        metric_name
    )

# -----------------------------
# 7. Flow Heterogeneity Index
# -----------------------------
print("\nComparison of Flow Heterogeneity Index:")
print(f"Validation: {flow_heterogeneity_valid:.4f}")
print(f"Predicted: {predicted_flow_heterogeneity:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure A_pred_denoised is a NumPy array
A_pred_denoised_np = A_np29_gravity29
A_valid_np = A_valid.cpu().detach().numpy()  # Convert A_valid to NumPy

A_valid_np = A_valid_np.squeeze(0)

# Compute metrics for the validation set
flow_metrics_valid, flow_heterogeneity_valid = compute_network_flow_metrics_extended(A_valid_np, filtered_countries)

# Compute metrics for the predicted graph
predicted_flow_metrics, predicted_flow_heterogeneity = compute_network_flow_metrics_extended(A_pred_denoised_np, filtered_countries)

# -----------------------------
# Helper function: Plot comparison for top 10 countries
# -----------------------------
def plot_comparison(original_data, predicted_data, title, ylabel):
    top_10 = sorted(original_data.items(), key=lambda x: x[1], reverse=True)[:10]  # Top 10 from original
    countries, original_values = zip(*top_10)  # Separate into country names and values
    predicted_values = [predicted_data[country] for country in countries]  # Get corresponding predicted values
    
    x = np.arange(len(countries))  # X positions
    width = 0.4  # Bar width

    plt.figure(figsize=(6, 3))
    plt.bar(x - width / 2, original_values, width, label="Validation")
    plt.bar(x + width / 2, predicted_values, width, label="Predicted")
    plt.xticks(x, countries, rotation=45, ha="right")
    plt.title(title)
    plt.ylabel(ylabel)
    plt.grid(False)  # Remove grid
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

# -----------------------------
# Correlation for each metric and plotting comparisons
# -----------------------------

metrics_to_compare = {
    "Inflow Hubs": "inflow_hubs_weighted",
    "Outflow Hubs": "outflow_hubs_weighted",
}

for metric_name, metric_key in metrics_to_compare.items():
    print(f"\nCorrelation for {metric_name}:")
    corr = np.corrcoef(
        list(flow_metrics_valid[metric_key].values()),
        list(predicted_flow_metrics[metric_key].values())
    )[0, 1]
    print(f"Correlation: {corr:.4f}")

    plot_comparison(
        flow_metrics_valid[metric_key],
        predicted_flow_metrics[metric_key],
        f"{metric_name} Comparison (Top 10)",
        metric_name
    )

# -----------------------------
# 7. Flow Heterogeneity Index
# -----------------------------
print("\nComparison of Flow Heterogeneity Index:")
print(f"Validation: {flow_heterogeneity_valid:.4f}")
print(f"Predicted: {predicted_flow_heterogeneity:.4f}")


In [ ]:
# Define feature groups for ablation
feature_groups = {
    "GDP": [0],
    "GINI": [1],
    "HDI": [2],
    "PTS": [3],
    "U5M": [4],
    "Language_Embeddings": list(range(5, 17))  # Treat all language embeddings as one chunk
}

# Set number of epochs for retraining
num_epochs_retrain = 500

# Dictionary to store performance drop for each feature group
feature_importance_retrain_500 = {}

# Retrain the baseline model (no masking) with 500 epochs
print("\nRetraining baseline model with 500 epochs...")
baseline_model_retrained = EGCU_H(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    num_gcn_layers=num_gcn_layers,
    num_heads=num_heads,
    attention=True
).to(device)

link_predictor_baseline_retrained = LinkPredictorMLP(input_dim=output_dim, weighted=False).to(device)
optimizer_baseline_retrained = optim.Adam(
    list(baseline_model_retrained.parameters()) + list(link_predictor_baseline_retrained.parameters()), 
    lr=learning_rate
)

# Train baseline retrained model
train(
    X_norm=X_norm, A_norm=A_norm, A_train=A_train,
    model=baseline_model_retrained, link_predictor=link_predictor_baseline_retrained,
    criterion=criterion, optimizer=optimizer_baseline_retrained,
    device=device, num_epochs=num_epochs_retrain, edge_subset=edge_subset,
    loss_weights=loss_weights, calc_reverse=calc_reverse,
    save_at=50, weighted=False
)

# Evaluate the retrained baseline model
_, baseline_metrics_retrained = eval_model(
    X_norm=X_norm1,
    A_norm=A_norm1,
    A_test=A_valid,
    model=baseline_model_retrained,
    link_predictor=link_predictor_baseline_retrained,
    device=device,
    loss_all=False,
    weighted=False
)
baseline_auc_retrained = baseline_metrics_retrained[1]  # AUC
print(f"Baseline AUC (retrained): {baseline_auc_retrained}")



In [ ]:
# Perform ablation study with retraining
for group_name, indices in feature_groups.items():
    print(f"\nExcluding {group_name} features and retraining with 500 epochs...")

    # Mask out the features for this group
    X_masked = X_norm.clone()
    X_masked[..., indices] = 0  # Mask the features for the group

    X_masked1 = X_norm1.clone()
    X_masked1[..., indices] = 0  # Mask the features for the group

    # Retrain the model with the masked features
    model_retrain_500 = EGCU_H(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        num_gcn_layers=num_gcn_layers,
        num_heads=num_heads,
        attention=True
    ).to(device)

    link_predictor_retrain_500 = LinkPredictorMLP(input_dim=output_dim, weighted=False).to(device)
    optimizer_retrain_500 = optim.Adam(
        list(model_retrain_500.parameters()) + list(link_predictor_retrain_500.parameters()), 
        lr=learning_rate
    )

    train(
        X_norm=X_masked, A_norm=A_norm, A_train=A_train,
        model=model_retrain_500, link_predictor=link_predictor_retrain_500,
        criterion=criterion, optimizer=optimizer_retrain_500,
        device=device, num_epochs=num_epochs_retrain, edge_subset=edge_subset,
        loss_weights=loss_weights, calc_reverse=calc_reverse,
        save_at=50, weighted=False
    )

    # Evaluate the retrained model
    _, retrain_metrics_500 = eval_model(
        X_norm=X_masked1,
        A_norm=A_norm1,
        A_test=A_valid,
        model=model_retrain_500,
        link_predictor=link_predictor_retrain_500,
        device=device,
        loss_all=False,
        weighted=False
    )

    retrain_auc_500 = retrain_metrics_500[1]  # AUC
    print(f"AUC after excluding {group_name} (500 epochs): {retrain_auc_500}")

    # Calculate the performance drop
    performance_drop = 100 * (baseline_auc_retrained - retrain_auc_500) / baseline_auc_retrained
    feature_importance_retrain_500[group_name] = performance_drop



In [ ]:
## feature_importance.keys()
key_mapping = {
    'Language_Embeddings' : 'Language', 
    'U5M' : 'Mortality under 5', 
    'PTS' : 'Political Terror Scale', 
    'HDI' : 'Human Development Index',
    'GINI' : 'GINI index', 
    'GDP' : 'GDP per capita'
}

feature_importance_retrain_500 = {
    key_mapping.get(key, key): value for key, value in feature_importance_retrain_500.items()
}

In [ ]:
# Reverse the order of features and their corresponding performance impacts
feature_names = list(feature_importance_retrain_500.keys())
performance_increases = list(feature_importance_retrain_500.values())

feature_names = feature_names[::-1]
performance_increases = performance_increases[::-1]

# Create the horizontal bar chart
plt.figure(figsize=(6, 4))
plt.barh(feature_names, performance_increases, color='skyblue', alpha=0.8)
#plt.ylabel('Feature Groups', fontsize=14)
plt.xlabel('Performance Decrease (%)', fontsize=14)
plt.title('Feature Ablation Performance Loss', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add a dotted line at the zero point
plt.axvline(x=0, color="gray", linestyle="--", linewidth=0.8)

# Remove grid
plt.grid(False)

# Despine by removing top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Display the bar chart
plt.tight_layout()
plt.show()
